In [36]:
import csv
from sklearn.metrics import accuracy_score

In [4]:
raw_x = []
y = []
dialogue_len = []
prob_reason = []
solution = []

with open('../features/positive_correlation_features.tsv', 'r') as rf:
    csv_reader = csv.reader(rf, delimiter='\t')
    for item in csv_reader:
        raw_x.append([float(i) for i in item[1:-1]])
        y.append(float(item[-1]))
        dialogue_len.append(float(item[1]))
        prob_reason.append(float(item[2]))
        solution.append(float(item[3]))        

In [14]:
import plotly.express as px
import pandas as pd

In [25]:
df = pd.DataFrame(dict(dialogue_length=dialogue_len, solution=solution,
                       outcome=y, reasoning=prob_reason))

# Use column names of df for the different parameters x, y, color, ...
fig = px.scatter(df, x="dialogue_length", y="outcome",
#                  color="outcome",
                  title="Dialogue_len to Outcome"
                )

fig.show()

In [26]:
fig = px.scatter(df, x="reasoning", y="outcome",
#                  color="outcome",
                  title="Reasoning to outcome"
                )

fig.show()

In [27]:
fig = px.scatter(df, x="solution", y="outcome",
#                  color="outcome",
                  title="Solution to outcome"
                )

fig.show()

In [58]:
simple_decision_tree_solution = []

LOWER = 0.1

UPPER = 0.32

for item in solution:
    if item > LOWER and item < UPPER:
        simple_decision_tree_solution.append(1)
    else:
        simple_decision_tree_solution.append(0)
accuracy_score(y, simple_decision_tree_solution)

0.68

In [28]:
fig = px.scatter(df, x="solution", y="reasoning",
                  color="outcome",
                  title="Solution to outcome"
                )

fig.show()

In [29]:
fig = px.scatter(df, x="solution", y="dialogue_length",
                  color="outcome",
                  title="Solution to outcome"
                )

fig.show()

In [30]:
fig = px.scatter(df, x="reasoning", y="dialogue_length",
                  color="outcome",
                  title="Solution to outcome"
                )

fig.show()

In [59]:
from sklearn.tree import DecisionTreeClassifier

In [74]:
clas = DecisionTreeClassifier(criterion='entropy', random_state=42, min_samples_leaf=3)

In [75]:
clas.fit(raw_x, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [76]:
from sklearn import tree

In [77]:
text_representation = tree.export_text(clas)
print(text_representation)

#feature_0 = dialogue_length
#feature_1 = probing reasoning
#feature_2 = solution 

|--- feature_2 <= 0.21
|   |--- feature_2 <= 0.18
|   |   |--- feature_2 <= 0.16
|   |   |   |--- feature_0 <= 28.50
|   |   |   |   |--- feature_0 <= 24.50
|   |   |   |   |   |--- feature_2 <= 0.13
|   |   |   |   |   |   |--- feature_2 <= 0.12
|   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |--- feature_2 >  0.12
|   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |--- feature_2 >  0.13
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |--- feature_0 >  24.50
|   |   |   |   |   |--- class: 0.0
|   |   |   |--- feature_0 >  28.50
|   |   |   |   |--- feature_1 <= 0.04
|   |   |   |   |   |--- class: 1.0
|   |   |   |   |--- feature_1 >  0.04
|   |   |   |   |   |--- class: 1.0
|   |   |--- feature_2 >  0.16
|   |   |   |--- class: 1.0
|   |--- feature_2 >  0.18
|   |   |--- class: 0.0
|--- feature_2 >  0.21
|   |--- feature_2 <= 0.33
|   |   |--- class: 1.0
|   |--- feature_2 >  0.33
|   |   |--- class: 1.0



In [87]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report
import numpy as np

In [92]:
loo = LeaveOneOut()

predicted = []
gold = []

np_x = np.array(raw_x)
np_y = np.array(y)

for train_index, test_index in loo.split(np_x):
    X_train, X_test = np_x[train_index], np_x[test_index]
    y_train, y_test = np_y[train_index], np_y[test_index]

    clas = DecisionTreeClassifier(criterion='entropy', random_state=42, min_samples_leaf=3)
    clas.fit(X_train, y_train)
    pred = clas.predict(X_test)
    # print("{} ::: {}".format(new_fit.predict_proba(X_test), y_test))
    predicted.append(pred)
    gold.append(y_test)

clas_rep = classification_report(gold, predicted)
print(clas_rep)

              precision    recall  f1-score   support

         0.0       0.32      0.43      0.36        14
         1.0       0.74      0.64      0.69        36

    accuracy                           0.58        50
   macro avg       0.53      0.53      0.53        50
weighted avg       0.62      0.58      0.60        50

